# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [ ]:
# Load dependencies
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2 as cv
import pickle
import time
from sklearn.utils import shuffle
from Utils.Perturb_Functions import *
%load_ext autoreload
%autoreload 2

In [ ]:
# Load pickled data
# TODO: Fill this in based on where you saved the training and testing data
# NB This repository does NOT contain either the regular data or the augmented data.
# Download/create these yourself and put them in the correct directory.

training_file =  './Data/train.p'
validation_file= './Data/valid.p'
testing_file =   './Data/test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height of the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [ ]:
### Replace each question mark with the appropriate value. 
### Use python, pandas or numpy methods rather than hard coding the results

# TODO: Number of training examples
n_train = X_train.shape[0]

# TODO: Number of validation examples
n_validation = X_valid.shape[0]

# TODO: Number of testing examples.
n_test = X_test.shape[0]

# TODO: What's the shape of a traffic sign image?
image_shape = X_train.shape[1:3]
H, W        = image_shape

# TODO: How many unique classes/labels there are in the dataset.
n_classes = len(np.unique(np.concatenate((y_train, y_valid, y_test))))

print("Number of training examples =", n_train)
print("Number of validation examples =", n_validation)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc. 

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections. It can be interesting to look at the distribution of classes in the training, validation and test set. Is the distribution the same? Are there more examples of some classes than others?

In [ ]:
### Data exploration visualization code goes here.
### Feel free to use as many code cells as needed.
# Visualizations will be shown in the notebook.
%matplotlib inline

# include a library with traffic sign ClassID -> SignName mappings 
import csv
sign_dict = {}
with open('signnames.csv', 'r') as csvfile:
    signnames = csv.reader(csvfile, delimiter=',', quotechar='|') #what does quotechar do??
    for i, row in enumerate(signnames):
        if (i > 0):
            print('ClassID ', ': '.join(row))
            sign_dict[row[0]] = row[1]

In [ ]:
# plot a random example. Rerun this cell for different examples
index = np.random.randint((X_train).shape[0])
plt.axis('off')
plt.imshow(X_train[index])
print('ClassID: {}, SignName: {}'.format(y_train[index], sign_dict[str(y_train[index])]))
print('current size: {}, original size: {}'.format((X_train).shape[1:3], (train['sizes'][index][:])))

In [ ]:
# plot the label distribution of the three different y_sets:

# precalculate normalized occurrences:
y_uniq = np.unique(np.concatenate((y_train, y_valid, y_test)))
y_tr_uniq, y_tr_occ = np.unique(y_train, return_counts=True)
y_va_uniq, y_va_occ = np.unique(y_valid, return_counts=True)
y_te_uniq, y_te_occ = np.unique(y_test,  return_counts=True)
assert (np.all(y_tr_uniq == y_va_uniq) & np.all(y_va_uniq == y_te_uniq)), \
            'Unique elements in the three sets are not the same!'
    
y_tr_occ_norm = y_tr_occ / np.max(y_tr_occ)
y_va_occ_norm = y_va_occ / np.max(y_va_occ)
y_te_occ_norm = y_te_occ / np.max(y_te_occ)
assert (np.sum(y_tr_occ)+np.sum(y_va_occ)+np.sum(y_te_occ)) == (n_train+n_validation+n_test), \
            'something wrong with occurance counts'
    
# distribution of the three sets, normalized comparison:
bar_width = 0.25
plt.figure(figsize=(16,3))
plt.axes().get_yaxis().set_visible(False)

plt.bar(y_uniq, y_tr_occ_norm, width=bar_width)
plt.bar(y_uniq + bar_width, y_va_occ_norm, width=bar_width)
plt.bar(y_uniq + 2*bar_width, y_va_occ_norm, width=bar_width)

plt.title('distribution of the three sets, normalized comparison:')
plt.legend(('train', 'valid', 'test'))
plt.savefig('data_class_distribution.jpg')
plt.show()

# separate distributions of the three unnormalized sets:)
print('separate distributions of the three unnormalized sets:')
fig, subplt = plt.subplots(3, 1, figsize=(16, 9), sharex=True)
subplt[0].hist(y_train, len(y_uniq), color='C0'); subplt[0].set_title('training set')
subplt[1].hist(y_valid, len(y_uniq), color='C1'); subplt[1].set_title('validation set')
subplt[2].hist(y_test,  len(y_uniq), color='C2'); subplt[2].set_title('test set')
plt.show()


----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play! 

With the LeNet-5 solution from the lecture, you should expect a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission. 

There are various aspects to consider when thinking about this problem:

- Neural network architecture (is the network over or underfitting?)
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Pre-process the Data Set (normalization, grayscale, etc.)

Minimally, the image data should be normalized so that the data has mean zero and equal variance. For image data, `(pixel - 128)/ 128` is a quick way to approximately normalize the data and can be used in this project. 

Other pre-processing steps are optional. You can try different techniques to see if it improves performance. 

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

In [ ]:
### Preprocess the data here. It is required to normalize the data. Other preprocessing steps could include 
### converting to grayscale, etc.
### Feel free to use as many code cells as needed.

# JUST LIKE WE ARE NOT ABLE TO LOOK AT THE TEST SET RESULTS BEFORE WE ATTAIN OUR FINAL MODEL,
# WE ARE NOT ALLOWED TO IMPORT FUTURE INFORMATION BY LOOKING AT THE VALIDATION OR TEST SET FOR NORMALIZING 
# Hence, normalize based on the training mean and standart deviation.
# Since our model expects normalized data input, we DO need to normalize the validation and test as well

tr_mu   = np.mean(X_train, axis=0)
tr_std  = np.std(X_train, axis=0)

X_tr = (X_train - tr_mu) / tr_std
X_va = (X_valid - tr_mu) / tr_std
X_te = (X_test  - tr_mu) / tr_std

# shuffle as in the class LeNet example:
X_tr, y_tr = shuffle(X_tr, y_train)
X_va, y_va = shuffle(X_va, y_valid)
X_te, y_te = shuffle(X_te, y_test)

In [ ]:
# store the preprocessing pipeline required data for future use:
np.save('Data/preproc_values.npy', [tr_mu, tr_std])

### Quick comment to Udacity

In [ ]:
# (Below) I used (a.o.) image shifting to augment the data size
# however, it seems this is already included in the original set:

x = 20
fig, subplt = plt.subplots(1, x, figsize=(20, 5))
for i in range(900, 900+x):
    subplt[i-900].imshow(X_train[i]); subplt[i-900].set_title('{}'.format(i)); subplt[i-900].axis('off')
plt.show()

# This also seems to include pictures of the same sign but from different angles
# Probably I should have also checked the database website better
# Still, I feel this should have been mentioned somewhere in the above explanation of the data base
# so as to prevent double augmentation techniques

### Data augmentation
According to the above mentioned [paper](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf) by LeCun.

There are other and more elaborate methods available as well, like Krizhevsky's [fancy PCA](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf) (color perturbation).

In [ ]:
# NB THIS CODE TAKES FOREVER. DONT RUN THIS CELL. SEE TWO CELL BELOW.
# augments the dataset by using 2 perturbing function
# two sets are made and pre saved (both including the labels):
# 
# X_augm_excl
# This set can be used as addition to the original set, as it doesnt include those samples
# It is 4 times as big, but any part of that can be added to the original X_tr set for flexible data sizes
#
# X_augm_incl
# Includes the original set, can be used stand alone
# 6 times as big as the original dataset, do not use only a part (we dont want to throw away original (best?) samples)

# scaling, one zoom in and one zoom out, according to the scale parameter (MUST: scale[0]<1 & scale[1]>1)
X_tr_a, y_tr_a = image_Scale(X_tr, y_tr, scale=[0.9, 1.1])

# cropping / shifting, this is not included, since it is already incorporated in the data set, see cell above 
#X_tr_a, y_tr_a = image_Shift(X_tr1_a, y_tr1_a, pixels=3) 

# rotation, angles in degrees (making sure there is no 0, NB left is untill, right is till)
angles         = np.append(np.random.randint(-15, 0), (np.random.randint(1, 16)))
X_tr_a, y_tr_a = image_Rotate(X_tr_a, y_tr_a, angles=angles)

# Perspective is a cool feature as well, but somewhat similar to rotation, not incorporated to limit the augmentation size
#X_tr, y_tr = image_Perspective(X_tr)

# Flipping not possible due to unsymmetric signs 
#img_flp,   lbl_flp  = image_Flip(img_shp,      lbl_shp)

# adding noise, could be useful for overfitting
#img_noi,   lbl_noi  = image_Noise(img_cpb,     lbl_cpb,  std)

# implementation of fancy PCA / color perturbation (not sure if correct hence not in use)
#img_cpb,   lbl_cpb  = image_Color_Perturb(img, lbl, no_c_pert)

# randomly shuffle again
#X_tr_a, y_tr_a = shuffle(X_tr_a, y_tr_a)

print('final training sample size:', X_tr_a.shape, 'label size:', y_tr_a.shape, '(axis 0 should be the same)')

### Quick check if augmentation went ok
Although very hard to see due to the normalization
Should we perhaps perform normalization after perturbation?

In [ ]:
index = np.random.randint((X_tr_a).shape[0])
print('index: {}'.format(y_tr_a[index]))

fig, subplt = plt.subplots(1,2, figsize=(16,3))
subplt[0].imshow(X_tr_a[index])
subplt[1].imshow(X_tr_a[index]*tr_std + tr_mu)
subplt[0].axis('off'); subplt[0].set_title('normalized augmented sample')
subplt[1].axis('off'); subplt[1].set_title('after warping the image back to un-normalization:')
plt.show()

# The perturbations can be clearly seen, but also can be seen that the images still retain their importent features
# what is werid however, is after un-normalizing any augmented image, it becomes rubbish:

# This is not the case for of course when un-normalizing 'un-augmented normalized images'
# Does this mean something is wrong?
# Most likely has to do with datatypes, plotting differently? (then should apply to un-augmented files as well..)

### Augmenting the data set might be computationally or time intensive. Thus for future usage save & load the set
Take care: saves the variable with the name X_tr_a, but loads an augmented set
- with the regular name X_tr if it includes the original set
- with the name X_au, if it does not include the original set (needs to be appended to X_tr still)

In [ ]:
save_load = 'load' # 'save' or load'

if save_load == 'save':
    pickle.dump([X_tr_a.astype('float32'), y_tr_a], open("Data/train_augm_excl.p", "wb"))
    
elif save_load == 'load':
    #X_au, y_au = pickle.load(open("Data/train_augm_excl.p", "rb"))#[:int(X_tr*2)] # adjust amount of extra samples
    X_tr, y_tr = pickle.load(open("Data/train_augm_incl.p", "rb"))

### Model Architecture

In [ ]:
### Define your architecture here.
### Feel free to use as many code cells as needed.

In [ ]:
# Initialization
from tensorflow.contrib.layers import flatten
import tensorflow as tf

# Placeholders
x_pl     = tf.placeholder(tf.float32, (None, H, W, X_tr[0].shape[-1])) # NOT SURE IF THE SHAPE REFERENCE WORKS LIKE THAT
y_pl     = tf.placeholder(tf.int32, (None))
y_onehot = tf.one_hot(y_pl, n_classes)
drop_pl  = tf.placeholder(dtype=tf.float32, shape=())
BN_pl    = tf.placeholder(dtype=tf.bool, shape=()) #BN placeholder to keep track of whether training or validating/testing
    
# Hyperparameters
EPOCHS     = 10
BATCH_SIZE = 128
DROPOUT    = 0.7    # KEEP PROBABILITY
LRN_RATE   = 0.001
N_CLASSES  = n_classes
BN_DECAY   = 0.999
BN_EPS     = 0.001


In [ ]:
# Helper functions

def maxpool(x, k=1):
    x = tf.nn.max_pool(x, [1, k, k, 1], [1, k, k, 1], 'VALID')
    return x

def conv2d_ReLU(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='VALID')
    x = tf.nn.bias_add(x, b)
    x = tf.nn.relu(x)
    return x

def conv2d_ReLU_maxp(x, W, b, strides=1, pool=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='VALID')
    x = tf.nn.bias_add(x, b)
    x = tf.nn.relu(x)
    x = maxpool(x, k=pool)
    return x

def conv2d_BN_ReLU(x, W, b, strides=1, training=True):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='VALID')
    x = tf.nn.bias_add(x, b)
    x = tf.contrib.layers.batch_norm(x, is_training=training)
    x = tf.nn.relu(x)
    return x

def conv2d_BN_ReLU_maxp(x, W, b, strides=1, BN_train=True, pool=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='VALID')
    x = tf.nn.bias_add(x, b)
    x = tf.contrib.layers.batch_norm(x, is_training=BN_train)
    x = tf.nn.relu(x)
    x = maxpool(x, k=pool)
    return x

def fc(x, W, b):
    x = tf.add(tf.matmul(x, W), b)
    return x

def fc_ReLU(x, W, b):
    x = tf.add(tf.matmul(x, W), b)
    x = tf.nn.relu(x)
    return x

def fc_ReLU_drop(x, W, b, keep_prob=1):
    x = tf.add(tf.matmul(x, W), b)
    x = tf.nn.relu(x)
    x = tf.nn.dropout(x, keep_prob)
    return x

def fc_BN_ReLU_drop(x, W, b, BN_train, keep_prob=1):
    x = tf.add(tf.matmul(x, W), b)
    x = tf.contrib.layers.batch_norm(x, is_training=BN_train)
    x = tf.nn.relu(x)
    x = tf.nn.dropout(x, keep_prob)
    return x

Xav_init = tf.contrib.layers.xavier_initializer()

In [ ]:
# Something is wrong with the namescoping here...

# with reuse=True for the variable scope, batch normalization wont initialize, throwing the error:
# "... does not exist, or was not created with tf.get_variable(). Did you mean to set reuse=None in VarScope?"

# with reuse=False however, I get the error on a second run:
# "... already exists, disallowed. Did you mean to set reuse=True in VarScope?"
def Tr_Sign_Net(x, BN_tr, dropout):    
    
    with tf.variable_scope('Conv1'): #Convolutional + max pool 32x32x3 -> 14x14x6
        #W = tf.get_variable("W", shape=[5, 5, 3, 6], initializer=tf.contrib.layers.xavier_initializer())
        W = tf.Variable(Xav_init([5, 5, 3, 6]))
        b = tf.Variable(tf.zeros([6]))
        x = conv2d_BN_ReLU_maxp(x, W, b, strides=1, BN_train=BN_tr, pool=2)

    with tf.variable_scope('Conv2'): #Convolutional + max pool 14x14x6 -> 5x5x16
        W = tf.Variable(Xav_init([5, 5, 6, 16]))
        b = tf.Variable(tf.zeros([16]))
        x = conv2d_BN_ReLU_maxp(x, W, b, strides=1, BN_train=BN_tr, pool=2)

    x   = flatten(x) # Flatten 5x5x16 -> 400
    
    with tf.variable_scope('FC3'): # Fully Connected + dropout 400 -> 120
        W = tf.Variable(Xav_init([400, 120]))
        b = tf.Variable(tf.zeros([120]))
        x = fc_BN_ReLU_drop(x, W, b, BN_train=BN_tr, keep_prob=dropout)
    
    with tf.variable_scope('FC4'): # Fully Connected 120 -> 84
        W = tf.Variable(Xav_init([120, 84]))
        b = tf.Variable(tf.zeros([84]))
        x = fc_BN_ReLU_drop(x, W, b, BN_train=BN_tr, keep_prob=dropout)
    
    with tf.variable_scope('FC5'): # Fully Connected 84 -> 43
        W = tf.Variable(Xav_init([84, 43]))
        b = tf.Variable(tf.zeros([43]))
        x = fc(x, W, b)
    
    logits = x
    
    return logits

In [ ]:
# similar architecture as above, but without variable name scoping. currently not in used
def Tr_Sign_Net(x, BN_tr, dropout):
    
    WC1 = tf.Variable(Xav_init([5, 5, 3, 6]))
    bC1 = tf.Variable(tf.zeros([6]))
    C1  = conv2d_BN_ReLU_maxp(x, WC1, bC1, strides=1, BN_train=BN_tr, pool=2)
    #C1  = conv2d_ReLU_maxp(x, WC1, bC1, strides=1, pool=2)

    WC2 = tf.Variable(Xav_init([5, 5, 6, 16]))
    bC2 = tf.Variable(tf.zeros([16]))
    C2  = conv2d_BN_ReLU_maxp(C1, WC2, bC2, strides=1, BN_train=BN_tr, pool=2)
    #C2  = conv2d_ReLU_maxp(C1, WC2, bC2, strides=1, pool=2)

    C2   = flatten(C2) # Flatten 5x5x16 -> 400
    
    WF3 = tf.Variable(Xav_init([400, 120]))
    bF3 = tf.Variable(tf.zeros([120]))
    F3  = fc_BN_ReLU_drop(C2, WF3, bF3, BN_train=BN_tr, keep_prob=dropout)
    #F3  = fc_ReLU_drop(C2, WF3, bF3, keep_prob=dropout)
    
    WF4 = tf.Variable(Xav_init([120, 84]))
    bF4 = tf.Variable(tf.zeros([84]))
    F4  = fc_BN_ReLU_drop(F3, WF4, bF4, BN_train=BN_tr, keep_prob=dropout)
    #F4  = fc_ReLU_drop(F3, WF4, bF4, keep_prob=dropout)
    
    WF5 = tf.Variable(Xav_init([84, 43]))
    bF5 = tf.Variable(tf.zeros([43]))
    F5  = fc(F4, WF5, bF5)
    
    logits = F5
    
    return logits

In [ ]:
# Some more tensorflow API

logits    = Tr_Sign_Net(x_pl, BN_pl, drop_pl)
CE_loss   = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_onehot, logits=logits))

# because of tf.contrib.layers.batch_norm apparently:
upd_ops   = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(upd_ops):
    optimizer = tf.train.AdamOptimizer(learning_rate = LRN_RATE).minimize(CE_loss)

cor_pred  = tf.equal(tf.argmax(logits, 1), tf.argmax(y_onehot, 1)) # correct predictions in array
accuracy  = tf.reduce_mean(tf.cast(cor_pred, tf.float32))          # correct predictions as accuracy
saver     = tf.train.Saver()

def evaluate(X_data, y_data):
    n        = len(X_data)
    full_acc = 0
    sess     = tf.get_default_session()
    
    for offset in range(0, n, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        batch_acc        = sess.run(accuracy, feed_dict={x_pl: batch_x, y_pl: batch_y, BN_pl: False, drop_pl: 1})
        full_acc        += (batch_acc * len(batch_x))
        
    return full_acc / n

### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [ ]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.

#UPDATE: STORE BEST VALIDATION ACCURACY EPOCH VARIABLES (NOT FINAL ONES)

In [ ]:
# Training, prints validation accuracy

print("Number of training examples =", len(X_tr))

tic = time.clock()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_tr)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_tr, y_tr = shuffle(X_tr, y_tr)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_tr[offset:end], y_tr[offset:end]
            sess.run(optimizer, feed_dict={x_pl: batch_x, y_pl: batch_y, BN_pl: True, drop_pl: DROPOUT})
            
        validation_accuracy = evaluate(X_va, y_va)
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
        
    saver.save(sess, './Models/sign_classifier')
    # save tr_mu & tr_std
    print("Model saved")
    print("Training time: {:.2f} sec".format(time.clock() - tic))

In [ ]:
# Final validation and test set evaluation
# Manualy replacing the saved model with the best validation accuracy (0.960) trained instance (in the 'back_up' folder)

with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('./Models'))
    
    tr_acc = evaluate(X_tr, y_tr[:,0])
    va_acc = evaluate(X_va, y_va)
    te_acc = evaluate(X_te, y_te)
    print("Training Accuracy = {:.3f}".format(tr_acc))
    print("Validation Accuracy = {:.3f}".format(va_acc))
    print("Test Accuracy = {:.3f}".format(te_acc))
    

In [ ]:
print(y_tr[:,0].shape)

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

### Predict the Sign Type for Each Image

In [ ]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.

In [ ]:
# load & show the images

# Except for a stop sign as bench mark for the farsi stop sign below, I've deliberately chosen rather hard examples
# Curious as to the performance with these tricky images
# I found these images online, which actually means:
# there is no guarentee they are indeed new and not already included in the dataset. But let's assume they're not

new_imgs = np.load('Data/new_online_images.npy')
fig, subplt = plt.subplots(1, 5, figsize=(10, 2), sharex=True)
subplt[0].imshow(new_imgs[0]); subplt[0].axis('off')
subplt[1].imshow(new_imgs[1]); subplt[1].axis('off')
subplt[2].imshow(new_imgs[2]); subplt[2].axis('off'); subplt[2].set_title('5 new german traffic sign images:')
subplt[3].imshow(new_imgs[3]); subplt[3].axis('off')
subplt[4].imshow(new_imgs[4]); subplt[4].axis('off')
plt.show()

# for fun, I have included this picture from my holiday in Iran
# let's see how well our model generalizes for similar yet different signs
farsi_stop = mpimg.imread('Data/Farsi_stop.jpg')
plt.axis('off')
plt.imshow(farsi_stop)
plt.title('Stop sign in farsi language')
plt.show()

In [ ]:
# preprocess the images

# right size
# (new images are already saved in the correct size)
farsi = cv.resize(farsi_stop, (W, H))

# normalize
mu, std = np.load('Data/preproc_values.npy')
X_new   = (new_imgs  - tr_mu) / tr_std
X_farsi = (farsi  - tr_mu) / tr_std


In [ ]:
# test the images

with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('./Models'))
    
    new_pred   = sess.run(tf.argmax(logits, 1), feed_dict={x_pl: X_new, BN_pl: False, drop_pl: 1})
    farsi_pred = sess.run(tf.argmax(logits, 1), feed_dict={x_pl: [X_farsi], BN_pl: False, drop_pl: 1})

    
# print the predictions above each image
fig, subplt = plt.subplots(1, 6, figsize=(20, 3), sharex=True)
for i in range(len(X_new)):
    subplt[i].imshow(new_imgs[i]); subplt[i].axis('off')
    subplt[i].set_title('prediction: {}'.format(sign_dict[str(new_pred[i])]))
subplt[5].imshow(farsi); subplt[5].axis('off')
subplt[5].set_title('predicted: {}'.format(sign_dict[str(farsi_pred[0])]))
plt.show()


### Analyze Performance

In [ ]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.


# As can be seen from above, not counting the farsi image,
# only the first and third new images are classified correctly, resulting in a 40% accuracy.
# This is fairly low but obviously due to the difficulty of the images
# (for example considering the fifth image, the circle is cut off above with a red line.
# It is not weird for the model to interpret the resulting shape as an upside down triangle: Yield)

# Interestingly enough, the stop sign in farsi language is classified correctly.
# perhaps the shape of a stop sign is more important than what is written on it
# or perhaps the model cannot 'read' actual languages, but only detect them
# hence it wouldn't make a difference for the model whether some written word is english or farsi

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tf.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.

k = 5

with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('./Models'))
    
    new_pred5  = sess.run(tf.nn.top_k(tf.nn.softmax(logits), k=k), feed_dict={x_pl: [X_farsi], BN_pl: False, drop_pl: 1})
    # farsi_pred = sess.run(tf.argmax(logits, 1), feed_dict={x_pl: [X_farsi], BN_pl: False, drop_pl: 1})

for i in range(len(X_new)):
    print('\nImage {} prediction:'.format(i+1))
    for j in range(k):
        print('With {:6.2f}% certainty: "{}"'.format(new_pred5[0][i][j]*100, sign_dict[str(new_pred5[1][i][j])]))

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

---

## Step 4 (Optional): Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it may be having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess, feed_dict={x_pl : image_input, BN_pl: False, drop_pl: 1})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")

In [ ]:
image_input    = [farsi]
tf_activation  = [var for var in tf.global_variables() if var.op.name=="Conv2/Variable_1"][0]
print(tf_activation)
#tf_activation  = tf.get_default_graph().get_tensor_by_name("Conv2/W")
activation_min = -1
activation_max = -1
plt_num        =  1

outputFeatureMap(image_input, tf_activation, activation_min, activation_max, plt_num)